# Script to generate predictions of KO strain design performance

## Detailed in "Integrated knowledge mining, genome-scale modeling, and machine learning for predicting *Yarrowia lipolytica* bioproduction".

### Description. 
Script takes a target compound, a list of reactions or genes to test for KO or OE, and generates predicted titers for each design. 

The default conditions for prediction are in glucose, with no prior genetic engineering. The prediction is obtained through generated machine-learning model detailed in the publication.

#### Procedure:
1. Read in data and constructs to screen from "Template_ComputationalDesign" spreadsheet.
2. Generate FBA features for the WT and each strain construct.
3. Predict the titer of each strain.
4. Output the results.

#### Inputs:
1. Supplemental Excel File 6- CSD Template.xlsx: Spreadsheet where the product, testing environmental conditons, and list of KO targets to screen are input.
      Supplemental Excel File 6- CSD Template.xslx
2. Data Encoding File 
      Supplemental Data File 2- DataCharateristics & Encoding.xlsx

#### Output:
1. titerPredictionsKO.csv: Spreadsheet containing a prediction of the WT strain titer, each KO strain titer, and the FBA predicted product yield and biomass growth rate.


#### Additional required scripts:
1. FBA_function_.py:
    Performs FBA feature generation and extraction
2. encodingFunction_.py:
    Encodes the data for input to the ML model
3. FBA_functionOE_.py:
    Performs Gene OE feature generation and extraction

### Libraries to import

In [1]:
import pandas as pd
import pickle
from collections import defaultdict
import warnings
import numpy as np
import os




# from FBA_function_ import FBA_FeatureExtraction
from encodingFunction_ import encodeTransform

### Ensure the spreadsheet is within the directory.

In [2]:
dir_path = os.path.dirname(os.path.realpath('Supplemental Excel File 6- CSD Template.xlsx'))
file_path = os.path.join(dir_path,'Supplemental Excel File 6- CSD Template.xlsx')

In [3]:
# reads in if you will performing knocksouts or overexpressions
inputKOorOE = pd.read_excel(file_path,sheet_name='KO_or_OE')



In [4]:
if inputKOorOE['Specify'][0]=='KO':  

# if perform_knockouts==1:
    #custom functions provided in the directory.
    from FBA_function_ import FBA_FeatureExtraction #KO
else:
    #custom functions provided in the directory for OE.
    from FBA_functionOE_ import FBA_FeatureExtraction #OE


In [5]:
#reads in the information from the datasheets
raw_construct = pd.read_excel(file_path,sheet_name='predictions',skiprows=range(2))
optKnockRxns = pd.read_excel(file_path,sheet_name='targetRxns')

optOERxns = optKnockRxns ## delete


# consolidate meta-information into usable features
data = raw_construct
optData = optKnockRxns
data['number_genes_mod'] = data.genes_modified_updated.apply(lambda x: x.count(';')+1 if isinstance(x,str) else 0)
data['number_genes_deleted'] = data.gene_deletion.apply(lambda x: x.replace(';','').count('1') if isinstance(x,str) else 0)
data['number_total_genes_overexp'] = data.gene_overexpression.apply(lambda x: x.replace(';','').count('1') if isinstance(x,str) else 0)
data['number_genes_het'] = data.heterologous_gene.apply(lambda x: x.replace(';','').count('1') if isinstance(x,str) else 0)

# hettemp1 = data.heterologous_gene#.apply(lambda x: x if isinstance(x,str) else 'NA')
hettemp1 = data.heterologous_gene.apply(lambda x: x if isinstance(x,str) else 'NA')
data.heterologous_gene
hettemp2 = hettemp1.str.split(';',expand=True)

# overexpressTemp1 = data.gene_overexpression.fillna('2')
overexpressTemp1 = data.gene_overexpression.apply(lambda x: x if isinstance(x,str) else 'NA')
overexpressTemp2 = overexpressTemp1.str.split(';',expand=True)
nativeGenes = overexpressTemp2[hettemp2=='0']

data['number_native_genes_overexp'] = nativeGenes.count(axis=1)


## FBA Modeling

In [6]:
##FBA modeling
#GSM to use, default is 'iYLI647'
FBA_models=['iYLI647']

#Extracted FBA features
if inputKOorOE['Specify'][0]=='KO':  

# if perform_knockouts==1:
    output = FBA_FeatureExtraction(data,optKnockRxns,FBA_models) #KO
else:
    output = FBA_FeatureExtraction(data,optKnockRxns,optKnockRxns,FBA_models) # OE



/home/jjczajka/Documents/GitHub/YlipCSD/PredictionFiles/FBA_functionOE_.py:46 UserWarning: DataFrame columns are not unique, some columns will be omitted.
/home/jjczajka/Documents/GitHub/YlipCSD/PredictionFiles/FBA_functionOE_.py:61 UserWarning: DataFrame columns are not unique, some columns will be omitted.


1.1398166174414033 iYLI647
Completed  1  overexpression simulations
0 OE failures
0 Prod failures
0 0 0 0 0 0 failure cases 1-6


In [7]:
output

,paper_number,blank1,cs1,cs_conc1,cs1_heatCombustion(kJ/mol),cs2,cs_conc2,cs2_heatCombustion(kJ/mol),reactor_type,rxt_volume,...,EMP_iYLI647,PPP_iYLI647,TCA_iYLI647,NADPH_iYLI647,ATP_iYLI647,PrdtFlux_iYLI647,PrdtYield_iYLI647,Biomass_iYLI647,O2Uptake_iYLI647,GlcUptake_iYLI647
0,1,NaN,10,20,34734.5,0,0,0,1,0.05,...,6.667322,7.343305,1.215066,15.982695,56.429296,0.210230,0.125476,0.854862,-13.207577,-10.0
1,1,NaN,10,20,34734.5,0,0,0,1,0.05,...,6.667322,7.343305,1.215066,15.982695,56.510524,0.210230,0.125476,0.854862,-13.207577,-10.0
2,1,NaN,10,20,34734.5,0,0,0,1,0.05,...,6.667322,7.343305,1.215066,15.982695,56.587441,0.210230,0.125476,0.854862,-13.207577,-10.0
3,1,NaN,10,20,34734.5,0,0,0,1,0.05,...,6.677714,7.280954,3.321615,15.985664,56.962038,0.205776,0.122817,0.854862,-13.483700,-10.0


## Encode data

In [8]:
#encode data, using output from FBA modeling section
encodedData = encodeTransform(output)

### Predict Machine learning trained model.

In [9]:
from ModulizeMLPredictions import perform_MLPrediction

In [10]:
MLOutput = perform_MLPrediction(encodedData,output)
MLOutput

,TiterPrediction(g/L),FBA predicted Biomass,FBA predicted Yield
0,0.008105,0.854862,0.125476
1,0.008105,0.854862,0.125476
2,0.008105,0.854862,0.125476
3,0.008986,0.854862,0.122817


In [ ]:
#Save file
MLOutput.to_csv('Screen.csv')